# Buiding Your First Machine Learning Model

In this Tutorial you will learn how to build a Convolutional Neural Network (CNN) to identify handwriting. It is an introductory tutorial that will teach you how to create the different layers in a CNN using the tensorflow library. 

## What is a Neural Network

A neural network (NN) is a machine learning model that is used to classify and predict a certain variable using a set of features in the data. Typically, it consists of an input layer that takes in the features, a number of hidden layers (more layer -> deep NN), and an oputput layer that returns the prediction. The layers in the NN are are composed of Neurons (aka Perceptrons), which are essentially a mathematical function in the from of `y = Wx + b` where W is the weight and b is the biase. Also, typically the output of a perceptron is passed to an [activation function which normalizes the result](https://en.wikipedia.org/wiki/Activation_function). 

A CNN is basically a NN with one or more of its hidden layers, typically first, being a [Convolution](https://en.wikipedia.org/wiki/Convolution). This allows the NN to extract features from input such as images.

![NN](https://i.imgur.com/xPMkf4i.png)

```
Baced on 'A deep MNIST classifier using convolutional layers'.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
```


## Imports

This tutorial uses [Tensorflow](https://www.tensorflow.org/), which is an open source library released by Google. 

In order to be able to build, test, and run a NN in Tensorflow, the following imports have to be used.
This also imports the MNIST data set (each point in the data set is a handwritten reprentation of the digits 0-9 in 784 pixels)

In [ ]:
"""A deep MNIST classifier using convolutional layers.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile
import argparse

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


## Building Blocks

In order to build a CNN, we need to define the basic building blocks: A convolution function, a max pool function, a weight, and a biase. These 4 functions will be later combined form the perceptrons and different layers in the CNN.

In [ ]:
# Convolution
def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# Max Pool 
def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Weight 
def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# Biase
def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## The Neural Network

This section of the code focuses on defining the CNN and the structure of the CNN. First the input is reshaped into a 28x28x1 tensor (array). 

In the first layer: the input is convoluted with by a weight and then the biase is added. The result is then normalised using the relu function. 

In the second layer: max pooling is applied to reduce the dimentionality after applying the convolution.

These two layers are repeated twice.

Then, then result of the second pooling layer is passed to a connected layer, then is passed through a dropout filter to reduce overfitting, and eventually the recuded-complexity 
model is passed to an output connected layer.

The output layer returns one of 10 classes pertaining to the digits 0-9.

In [ ]:
def deepnn(x):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  return y_conv, keep_prob


## Building the CNN

Having defied the CNN, we then build it.

First: the data set is imported into the environment and stored in the variable `mnist`.

Second: the input and outpu are defined as `x` and `_y` and the CNN is applied to `x`.

Then, the cross entropy is defined as the error function to be reduced and the training method is defined as well.

Finally, the accuracy is calculated and the Tensorflow graph is defined.

In [ ]:
# Import data
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Build the graph for the deep net
y_conv, keep_prob = deepnn(x)

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)

with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

graph_location = tempfile.mkdtemp()
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())

## Training the CNN

The last step is doing the actual training of the CNN. 

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

## Conclusion

Having completed the steps above, you can now deploy models from other frameworks such as sklearn and sparkml on DSX. You can then try to setup python and tensorflow on your personal computer and run this Python script and compare the results. It might take hours, maybe days to train depending on the performance of your machine.